In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from IPython.display import display
from IPython.display import display_html
from sklearn.feature_selection import RFECV
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [3]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
df_data  =train.append(test)


In [6]:
df_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [12]:
df_data.nunique()
df_data["Ticket"].describe()


count         1309
unique         929
top       CA. 2343
freq            11
Name: Ticket, dtype: object

###  將票價分組

In [ ]:
df_data['Fare'] = df_data['Fare'].fillna(df_data['Fare'].median())
df_data['FareBin_5']   = pd.qcut(df_data['Fare'],5)

### 給予區間標籤 and 性別更改回int64 EX male->1 

In [ ]:
label = LabelEncoder()
df_data['FareBin_Code_5'] = label.fit_transform(df_data['FareBin_5'])
df_data['Sex']            = label.fit_transform(df_data['Sex'])

In [ ]:
X_train = df_data[:len(train)].iloc[:,[2,4,13]]
X_test  = df_data[len(train):].iloc[:,[2,4,13]]
Y_train = df_data[:len(train)].iloc[:,1].astype('int64')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#RFC = RandomForestClassifier(random_state=2,n_estimators=250,min_samples_split=20,oob_score=True)
RFC   = RFECV(RandomForestClassifier(n_estimators=250,min_samples_split=20),cv=10,n_jobs=-1)
RFC.fit(X_train, Y_train)
Y_pred = RFC.predict(X_test)
randomfroest_score = round(RFC.score(X_train, Y_train) * 100, 2)
randomfroest_score

In [ ]:
index = pd.RangeIndex(start =892, stop =1310 , step =1)
submission = pd.DataFrame(
{"PassengerId" : index ,
 "Survived"    : Y_pred}
)
submission.to_csv("titanic90.csv", index= False)